## Connecting to PT NAD

ptnad-client allows you to make requests to the system using various authentication methods.

### Local Connection

For quick testing and debugging, you can use a local connection. The default credentials are login "administrator" and password "Administr@t0r".

In [ ]:
from ptnad import PTNADClient

# Connecting to a local PT NAD instance (example for a test environment)
client = PTNADClient("https://1.3.3.7", verify_ssl=False)

# Setting username and password
client.set_auth(username="administrator", password="Administr@t0r")

# Logging into the service
client.login()

### Connecting via SSO

To connect via SSO, you need to use auth_type="sso" along with parameters that can be obtained from the /opt/ptsecurity/etc/iam_cookie.json file on the PT NAD server.

Here’s an example structure from the iam_cookie.json file:

```bash
cat /opt/ptsecurity/etc/iam_cookie.json
```

```json
{
    "iam": {
        "id": "ptnad",
        "name": "PT NAD",
        "redirect_url": "https://1.3.3.7",
        "secret": "11111111-abcd-asdf-12334-0123456789ab",
        "token": "your_token",
        "token_type": "reference",
        "url": "https://siem.example.local:3334",
        "verify": false
    },
    "tms": {
        "app_id": "your_app_id",
        "display_name": "PT NAD",
        "endpoint": "https://1.3.3.7",
        "request_id": "your_request_id",
        "token": "your_token",
        "url": "https://siem.example.local:3334"
    }
}
```

To establish the connection, extract the values from the "id", "secret", and "url" keys.
Set auth_type to "sso", provide your login and password, and use the remaining fields from iam_cookie.json, where:
- client_id = id
- client_secret = secret
- sso_url = url

As a result we get:

```python
client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(
    auth_type="sso",
    username="user",
    password="pass",
    client_id="ptnad",
    client_secret="11111111-abcd-asdf-12334-0123456789ab",
    sso_url="https://siem.example.local:3334"
)
client.login()
```

Here is a link to the source code where you can learn more about the authentication mechanisms: [Code Reference](https://security-experts-community.github.io/ptnad-client/reference)

### Examples of using the library

#### BQL Queries

This example demonstrates working with custom filters stored in the file "docs\data\filters.json" and executing BQL queries to PT NAD. For this example, we'll select only two fields: source IP address (src.ip) and destination IP address (dst.ip):
- Connect locally to PT NAD.
- Load filters from the filters.json file, where each filter contains a BQL fragment, name, and category (with language selection capability).
- Form a base query with a time range (last 3 days).
- Combine the base query with filters.
- Group queries by categories.
- Execute each query and display results with:
    - filter category;
    - filter name;
    - complete BQL query;
    - results.

For more detailed information about possible filtering parameters and their values, please refer to [PT NAD Documentation](https://help.ptsecurity.com/en-US/projects/nad/12.2/help/7083891595)

In [ ]:
import json
from datetime import datetime, timedelta
from ptnad import PTNADClient

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

with open("../data/filters.json", "r", encoding="utf-8") as f:
    data = json.load(f)

now = datetime.now()
start_time = (now - timedelta(days=3)).strftime("%Y-%m-%dT%H:%M:%S")
end_time = now.strftime("%Y-%m-%dT%H:%M:%S")

base_query = f"SELECT src.ip, dst.ip FROM flow WHERE start > '{start_time}' AND end < '{end_time}'"

grouped_queries = {}

for item in data["filters"]:
    category = item["category"]["en"]
    name = item["name"]["en"]
    filter_query = item["filter"].strip()
    compiled_filter = client.filters.compile(filter_query)

    if filter_query:
        full_query = f"{base_query} AND {compiled_filter}"
        if category not in grouped_queries:
            grouped_queries[category] = []
        grouped_queries[category].append((name, full_query))

for category, filters in grouped_queries.items():
    print(f"\n▶️ Category: {category}")
    for name, query in filters:
        result = client.bql.execute(query)
        print(f"\n🔹 {name}\n{query}")
        print("Results:")
        if isinstance(result, list) and result:
            for i, row in enumerate(result, 1):
                print(f"{i}. {row}")
        else:
            print("No results.")

#### Monitoring

Getting the general status of the system:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.monitoring import MonitoringAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

monitoring = MonitoringAPI(client)

status = monitoring.get_status()

print(f"Status: {status.status}")
for problem in status.problems:
    print(f"{problem}")

Getting a list of all triggers:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.monitoring import MonitoringAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

monitoring = MonitoringAPI(client)

triggers = monitoring.get_triggers()

for trig in triggers:
    print(f"\nTrigger ID: {trig.id}")
    print(f"Type: {trig.type}")
    print(f"Status: {trig.status}")
    print(f"Last update: {trig.updated}")

#### Replists

Getting all lists:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

lists = replists.get_all_lists()
print(f"Total number: {len(lists)}")

for r in lists:
    print(f"- [{r['id']}] {r['name']} ({r['type']}, items: {r['items_count']})")


Search by keyword:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

results = replists.get_lists(search="APT")

for r in results:
    print(f"{r['name']}")

Creating a new list:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

new_list = replists.create_list(
    name="test_list_01",
    type="ip",
    color="1",
    description="Test list of IP addresses",
    content="1.1.1.1\n2.2.2.2"
)
print(f"The list has been created: {new_list['id']}")

List update:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

updated = replists.update_list(list_id=new_list["id"], description="Updated IP test list")
print("Description updated:", updated["description"])

Deleting a list:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.replists import RepListsAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

replists = RepListsAPI(client)

replists.delete_list(list_id=new_list["id"])
print("The list has been removed")

#### Signatures

Getting all signature classes:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.signatures import SignaturesAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

signatures = SignaturesAPI(client)

classes = signatures.get_classes()

for cls in classes:
    print(f"{cls['title']} (priority: {cls['priority']})")


Getting complete information on one signature:

In [ ]:
from ptnad import PTNADClient
from ptnad.api.signatures import SignaturesAPI

client = PTNADClient("https://1.3.3.7", verify_ssl=False)
client.set_auth(username="administrator", password="Administr@t0r")
client.login()

signatures = SignaturesAPI(client)

rules = signatures.get_rules(search="ftp", cls="Network scan")

for r in rules:
    print(f"\nRule #{r['sid']} - {r['msg']}")
    print(f"Vendor: {r['vendor']}, Enabled: {r['enabled']}")